# <center>  Классификация твитов. Базанов Дмитрий Б03-903 </center>

In [1]:
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.model_selection import train_test_split

In [2]:
raw_data = pd.read_csv('Tweets.csv')
print('shape:', raw_data.shape)

shape: (14640, 15)


In [3]:
raw_data.head(3)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)


Выделим уникальные названия компаний и юзернеймы:

In [4]:
usernames = raw_data['name'].apply(lambda x: x.lower()).unique()
airlines = raw_data['airline'].apply(lambda x: x.lower()).unique()

Сразу удалим признаки, которые нам не нужны для решения задачи по самим твитам


In [5]:
data = raw_data.drop(['tweet_location', 'user_timezone', 'tweet_coord', 'negativereason', 'name', 'retweet_count',
                     'airline_sentiment_gold', 'negativereason_gold', 'tweet_created', 'tweet_id', 'airline'], axis=1)

In [6]:
data.head()

,airline_sentiment,airline_sentiment_confidence,negativereason_confidence,text
0,neutral,1.0000,NaN,@VirginAmerica What @dhepburn said.
1,positive,0.3486,0.0000,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,NaN,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,0.7033,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,1.0000,@VirginAmerica and it's a really big bad thing...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   airline_sentiment             14640 non-null  object 
 1   airline_sentiment_confidence  14640 non-null  float64
 2   negativereason_confidence     10522 non-null  float64
 3   text                          14640 non-null  object 
dtypes: float64(2), object(2)
memory usage: 457.6+ KB


## Препроцессинг:

В данных есть пропуске в поле 'negativereason_confidence'. Рассмотрим, как распределён этот признак среди 'airline_sentiment'	

In [8]:
data.groupby('airline_sentiment').mean()

,airline_sentiment_confidence,negativereason_confidence
airline_sentiment,,
negative,0.933365,0.731769
neutral,0.823303,0.000000
positive,0.872039,0.000000


Видим, что если отзыв положительный или нейтральный, то уверенность в негативности отзыва всегда равна нулю. Посмотрим, сколько негативных отзывов с пустым полем:

In [9]:
data[['negativereason_confidence', 'airline_sentiment']]\
[(data.negativereason_confidence.isnull()) & (data.airline_sentiment != 'negative')].dropna().shape[0]

0

То есть все поля, которые соответствуют отрицательному отзыву, заполнены, а если средне значение положительных и нейтральных отзывов равно нулю, то мы можем заполнить все пустые значений нулями и не изменить среднее.

In [10]:
data['negativereason_confidence'] = data['negativereason_confidence'].fillna(0)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   airline_sentiment             14640 non-null  object 
 1   airline_sentiment_confidence  14640 non-null  float64
 2   negativereason_confidence     14640 non-null  float64
 3   text                          14640 non-null  object 
dtypes: float64(2), object(2)
memory usage: 457.6+ KB


In [12]:
data.groupby('airline_sentiment').mean()

,airline_sentiment_confidence,negativereason_confidence
airline_sentiment,,
negative,0.933365,0.731769
neutral,0.823303,0.000000
positive,0.872039,0.000000


Теперь выделим целевой признак

In [13]:
y = data.replace({'airline_sentiment': {'positive': 1, 'negative': -1, 'neutral': 0}})['airline_sentiment']


In [14]:
y.head()

0    0
1    1
2    0
3   -1
4   -1
Name: airline_sentiment, dtype: int64

In [15]:
X = data['text']

In [16]:
X.head()

0                  @VirginAmerica What @dhepburn said.
1    @VirginAmerica plus you've added commercials t...
2    @VirginAmerica I didn't today... Must mean I n...
3    @VirginAmerica it's really aggressive to blast...
4    @VirginAmerica and it's a really big bad thing...
Name: text, dtype: object

Напишем функцию для обработки каждого отдельного твита на предмет Никнеймов и прочего ненужного шума

In [17]:
import re
import nltk

In [18]:
def tweet_to_words(tweet):
    tweet = re.sub(r'@\w+', '', tweet)
    letters = re.sub("[^a-zA-Z]", " ", tweet)
    words = letters.lower().split()
    words = [x for x in words if not x in airlines]
    return (' '. join(words))

In [19]:
data['clean_tweet']=data['text'].apply(lambda x: tweet_to_words(x))

In [20]:
data.head()

,airline_sentiment,airline_sentiment_confidence,negativereason_confidence,text,clean_tweet
0,neutral,1.0000,0.0000,@VirginAmerica What @dhepburn said.,what said
1,positive,0.3486,0.0000,@VirginAmerica plus you've added commercials t...,plus you ve added commercials to the experienc...
2,neutral,0.6837,0.0000,@VirginAmerica I didn't today... Must mean I n...,i didn t today must mean i need to take anothe...
3,negative,1.0000,0.7033,@VirginAmerica it's really aggressive to blast...,it s really aggressive to blast obnoxious ente...
4,negative,1.0000,1.0000,@VirginAmerica and it's a really big bad thing...,and it s a really big bad thing about it


## Применим лемматизацию

In [21]:
from nltk.stem import WordNetLemmatizer 

In [22]:
def lem(tweet):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(tweet)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return(lemmatized_output)

Применим для каждого твита:

In [23]:
data['lem_clean_tweet']=data['clean_tweet'].apply(lambda x: lem(x))

In [24]:
data[['clean_tweet', 'lem_clean_tweet']].head()

,clean_tweet,lem_clean_tweet
0,what said,what said
1,plus you ve added commercials to the experienc...,plus you ve added commercial to the experience...
2,i didn t today must mean i need to take anothe...,i didn t today must mean i need to take anothe...
3,it s really aggressive to blast obnoxious ente...,it s really aggressive to blast obnoxious ente...
4,and it s a really big bad thing about it,and it s a really big bad thing about it


## Так же применим стемминг:

In [25]:
from nltk.stem import PorterStemmer

In [26]:
def stem(tweet):
    porter = PorterStemmer()
    word_list = nltk.word_tokenize(tweet)
    porter_output = ' '.join([porter.stem(w) for w in word_list])
    return(porter_output)

In [27]:
data['stem_lem_clean_tweet']=data['lem_clean_tweet'].apply(lambda x: stem(x))

In [28]:
data[['lem_clean_tweet', 'stem_lem_clean_tweet']].head()

,lem_clean_tweet,stem_lem_clean_tweet
0,what said,what said
1,plus you ve added commercial to the experience...,plu you ve ad commerci to the experi tacki
2,i didn t today must mean i need to take anothe...,i didn t today must mean i need to take anoth ...
3,it s really aggressive to blast obnoxious ente...,it s realli aggress to blast obnoxi entertain ...
4,and it s a really big bad thing about it,and it s a realli big bad thing about it


In [29]:
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [30]:
count_vect = CountVectorizer(stop_words='english', analyzer='word')
train_features = count_vect.fit_transform(train['stem_lem_clean_tweet'])
test_features = count_vect.transform(test['stem_lem_clean_tweet'])
train_features.shape, test_features.shape

((11712, 8705), (2928, 8705))

## Подберём модель на данных после всего лемматизации и стемминга

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, accuracy_score, \
    precision_score, make_scorer, f1_score
import xgboost as xgb

Функция для вывода всех нужных метрик:

In [32]:
def scores(true, pred):
    print('accuracy_score', round(accuracy_score(true, pred), 3))
    print('roc_auc_score', round(roc_auc_score(true,
        preds_proba, multi_class='ovr', average="weighted"), 3))
    print('recall_score', round(recall_score(true, pred, average="weighted"), 3))
    print('precision_score', round(precision_score(true, pred, average="weighted"), 3))

Параметры по умолчанию:

In [33]:
%%time
models = [
    LogisticRegression(random_state=42, max_iter=200),
    xgb.XGBClassifier(random_state=42),
    RandomForestClassifier(random_state=42)
]

for model in models:
    model.fit(train_features, train['airline_sentiment'])
    pred = model.predict(test_features)
    preds_proba = model.predict_proba(test_features)
    print('=========', model.__class__.__name__, '=========')
    scores(test.airline_sentiment, pred)

========= LogisticRegression =========
accuracy_score 0.799
roc_auc_score 0.899
recall_score 0.799
precision_score 0.794
========= XGBClassifier =========
accuracy_score 0.785
roc_auc_score 0.89
recall_score 0.785
precision_score 0.776
========= RandomForestClassifier =========
accuracy_score 0.768
roc_auc_score 0.871
recall_score 0.768
precision_score 0.759
CPU times: user 37.9 s, sys: 19.1 s, total: 57 s
Wall time: 17.3 s


По сетке LogisticRegression

In [37]:
%%time
params_lr = {
    'max_iter': [200, 350, 500],
    'C': [0.01, 0.5, 1.0],
    
}

logregrGrid = GridSearchCV(LogisticRegression(random_state=42), params_lr, scoring=make_scorer(f1_score, average="weighted"), n_jobs=-1)
logregrGrid.fit(train_features, train['airline_sentiment'])
scores(test['airline_sentiment'], logregrGrid.predict(test_features))

accuracy_score 0.802
roc_auc_score 0.871
recall_score 0.802
precision_score 0.796
CPU times: user 11.7 s, sys: 12.3 s, total: 24 s
Wall time: 19.9 s


По сетке XGBoost

In [46]:
%%time
params_xgb = {
    'min_child_weight': [1, 5],
    'gamma': [1, 2],
    'subsample': [0.3, 0.7],
    'eta': [0.1, 0.3],
    'max_depth': [9, 15]
}

XGBGrid = GridSearchCV(xgb.XGBClassifier(random_state=42), params_xgb, 
                       scoring=make_scorer(f1_score, average="weighted"), n_jobs=-1, verbose=10)
XGBGrid.fit(train_features, train['airline_sentiment'])
scores(test['airline_sentiment'], XGBGrid.predict(test_features))


Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  5.1min finished


accuracy_score 0.795
roc_auc_score 0.871
recall_score 0.795
precision_score 0.79
CPU times: user 25.5 s, sys: 283 ms, total: 25.8 s
Wall time: 5min 11s


По сетке RandomForest

In [57]:
%%time
params_rf = {
    'n_estimators': [200,300, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [2, 5, 10],
    'criterion':['gini', 'entropy']
}

RFGrid = GridSearchCV(RandomForestClassifier(random_state=42), params_rf, 
                      scoring=make_scorer(f1_score, average="weighted"), n_jobs=-1, verbose=10)
RFGrid.fit(train_features, train['airline_sentiment'])
scores(test['airline_sentiment'], RFGrid.predict(test_features))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   41.5s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   51.3s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1

accuracy_score 0.645
roc_auc_score 0.871
recall_score 0.645
precision_score 0.416
CPU times: user 1.39 s, sys: 123 ms, total: 1.51 s
Wall time: 1min 41s


/home/zugzvangg/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Лучшие, довольно хорошие метрики, даёт логистическая регрессия. Поиск по сетке не очень значительно, но улучшает метрики. При использовании RandomForest без сетки, он на удивление хуже отрабатывает, нежели без сетки. В целом метрики выходят нормальными, если использовать XGBoost или Регрессию. RandomForest плохо подходит для этой задачи.